In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import PIL.Image

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.8.0'

In [5]:
import vgg16

In [6]:
vgg16.data_dir = '.'

In [8]:
vgg16.maybe_download()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [9]:
def load_image(filename, max_size=None):
    image = PIL.Image.open(filename)
    
    if max_size is not None:
        # Rescale factor
        factor = max_size / np.max(image.size)
        
        # Scale the image
        size = np.array(image.size) * factor
        
        # Convert floating point size to int
        size = size.astype(int)
        
        # Resize the image
        image = image.resize(size, PIL.Image.LANCZOS)
        
    # Convert to numpy floating-point array
    return np.float32(image)

In [10]:
def save_image(image, filename):
    """
        Save image as jpg
    """
    image = np.clip(image, 0.0, 255.0)
    
    # Convert to bytes
    image = image.astype(np.uint8)
    
    # Write the image-file in JPG
    with open(filename, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')

In [11]:
def plot_image_big(image):
    """
        Plots a large image
    """
    image = np.clip(image, 0.0, 255.0)
    
    image = image.astype(np.uint8)
    
    display(PIL.Image.fromarray(image))

In [12]:
def plot_images(content_image, style_image, mixed_image):
    """
        Displays the content, style and mixed images together
    """
    # Create figure with sub-plots
    fig, axes = plt.subplots(1, 3, figsize=(10, 10))
    
    # Adjust vertical spacing
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    
    # Use interpolation to smooth pixels
    smooth = True
    
    # Interpolation type
    if smooth:
        interpolation = 'sinc'
    else:
        interpolation = 'nearest'
        
    # Plot normalized pixel values
    # Content-image
    ax = axes.flat[0]
    ax.imshow(content_image / 255.0, interpolation=interpolation)
    ax.set_xlabel("Content")
    
    # Mixed-image
    ax = axes.flat[1]
    ax.imshow(mixed_image / 255.0, interpolation=interpolation)
    ax.set_xlabel('Mixed')
    
    # Style-image
    ax = axes.flat[2]
    ax.imshow(style_image / 255.0, interpolation=interpolation)
    ax.set_xlabel('Style')
    
    # Remove ticks
    for ax in axes.flat:
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.show()

### Loss functions

In [13]:
def mean_squared_error(a, b):
    return tf.reduce_mean(tf.square(a - b))

In [14]:
def create_content_loss(session, model, content_image, layer_ids):
    # Create feed-dict for content-image
    feed_dict = model.create_feed_dict(image=content_image)
    
    # Get references to tensors for given layers
    layers = model.get_layer_tensors(layer_ids)
    
    # Calculate output of those layers for content-image
    values = session.run(layers, feed_dict=feed_dict)
    
    with model.garph.as_default():
        layer_losses = []
        
        for value, layer in zip(values, layers):
            value_const = tf.constant(value)
            
            loss = mean_squared_error(layer, value_const)
            
            layer_losses.append(loss)
        
        total_loss = tf.reduce_mean(layer_losses)
    return total_loss

In [15]:
def gram_matrix(tensor):
    """
        Finds which features activate simultaneously
    """
    num_channels = int(shape[3])
    
    matrix = tf.reshape(tensor, shape=[-1, num_channels])
    
    gram = tf.matmul(tf.transpose(matrix), matrix)
    
    return gram

In [16]:
def create_style_loss(session, model, style_image, layer_ids):
    feed_dict = model.create_feed_dict(image=style_image)
    
    layers = model.get_layer_tensors(layer_ids)
    
    with model.graph.as_default():
        gram_layers = [gram_matrix(layer) for layer in layers]
        
        values = session.run(gram_layers, feed_dict=feed_dict)
        
        layer_losses = []
        
        for value, gram_layer in zip(values, gram_layers):
            value_const = tf.constant(value)
            
            loss = mean_squared_error(gram_layer, value_const)
            
            layer_losses.append(loss)
        
        total_loss = tf.reduce_mean(layer_losses)
    return total_loss

In [17]:
def create_denoise_loss(model):
    loss = tf.reduce_sum(tf.abs(model.input[:, 1:, :, :] - model.input[:, :-1, :, :])) + \
            tf.reduce_sum(tf.abs(model.input[:, :, 1:, :] - model.input[:, :, :-1, :]))
    
    return loss

In [18]:
def style_transfer(content_image, style_image,
                  content_layer_ids, style_layer_ids,
                  weight_content=1.5, weight_style=10.0,
                  weight_denoise=0.3,
                  num_iterations=120, step_size=10.0):
    # Create instance of VGG model
    model = vgg16.VGG16()
    
    session = tf.InteractiveSession(graph=model.graph)
    
    print("Content layers:")
    print(model.get_layer_names(content_layer_ids))
    print()
    
    print("Style layers:")
    print(model.get_layer_names(style_layer_ids))
    print()
    
    loss_content = create_content_loss(session=session, model=model, content_image=content_image, 
                                       layer_ids=content_layer_ids)
    
    loss_style = create_style_loss(session=session, model=model, style_image=style_image, 
                                   layer_ids=content_layer_ids)
    
    loss_denoise = create_denoise_loss(model)
    
    adj_content = tf.Variable(1e-10, name='adj_content')
    adj_style = tf.Variable(1e-10, name='adj_style')
    adj_denoise = tf.Variable(1e-10, name='adj_denoise')
    
    update_adj_content = adj_content.assign(1.0 / (loss_content + 1e-10))
    update_adj_style = adj_style.assign(1.0 / (loss_style + 1e-10))
    update_adj_denoise = adj_denoise.assign(1.0 / (loss_denoise + 1e-10))
    
    loss_combined = weight_content * adj_content * loss_content + \
                    weight_style * adj_style * loss_style + \
                    weight_denoise * adj_denoise * loss_denoise
    
    gradient = tf.gradients(loss_combined, model.input)
    
    run_list = [gradient, update_adj_content, update_adj_style, update_adj_denoise]
    
    mixed_image = np.random.rand(*content_image.shape) + 128
    
    for i in range(num_iterations):
        feed_dict = model.create_feed_dict(image=mixed_image)
        
        grad, adj_content_val, adj_style_val, adj_denoise_val = session.run(run_list, feed_dict=feed_dict)
        
        grad = np.squeeze(grad)
        
        step_size_scaled = step_size / (np.std(grad) + 1e-8)
        
        mixed_image -= grad * step_size_scaled
        
        mixed_image = np.clip(mixed_image, 0.0, 255.0)
        
        print(". ", end="")
        
        if (i % 10 == 0) or (i == num_iterations - 1):
            print()
            print("Iteration:", i)
            
            msg = "Weight Adj. for Content: {0:.2e}, Style: {1:.2e}, Denoise: {2:.2e}"
            print(msg.format(adj_content_val, adj_style_val, adj_denoise_val))
            
            plot_images(content_image=content_image, style_image=style_image, mixed_image=mixed_image)
    print()
    print("Final image")
    plot_image_big(mixed_image)
    
    session.close()
    
    return mixed_image

In [19]:
content_filename = 'willy_wonka_old.jpg'
content_image = load_image(content_filename, max_size=None)

In [20]:
style_filename = "style2.jpg"
style_image = load_image(style_filename)

In [21]:
content_layer_ids = [4]

In [23]:
style_layer_ids = list(range(4))

In [ ]:
%%time
img = style_transfer(content_image=content_image,
                    style_image=style_image,
                    content_layer_ids=content_layer_ids,
                    style_layer_ids=style_layer_ids,
                    weight_content=1.5,
                    weight_style=10.0,
                    weight_denoise=0.3,
                    num_iterations=60,
                    step_size=10.0)